1\. Write a function that converts number representation (bin<->dec<->hex)

In [1]:
def bin2hex(strbin):
    return hex(int(strbin,2))

def hex2bin(fhex):
    return bin(int(fhex,16))

a = 345

bin2hex(bin(a))
hex2bin(hex(a))


'0b101011001'

2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [1]:
b = 2383116104


def floatingpoint(a):
    # to obtain the exponent value, we have two extract the bits from the 24th position, to the 24+8th,
    # it can be done by logically compare the number a to 2139095040 (1111111100000000000000000000000 in binary)
    temp = ( a & 2139095040 ) - 127
    exponent = 2**( temp )
    
    # similar reasoning was done to obtain the mantissa
    temp = ( a & 8388607 )
    mantissa = 1 + ( temp )

    if ( (a & 2147483648) == 2147483648 ):
        return (- mantissa * exponent)
    else:
        return (mantissa * exponent)

print(floatingpoint(b))

# didn't manage to get a result from this specific cell, the code is stuck on In [*] for some reason

KeyboardInterrupt: 

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [ ]:
def findmax():
    a = 1
    b = 2
    
    while( b > a ):
        a += 1
        b = 2 * b
    
    print(a)
    
findmax()

4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [3]:
precision = 1
i = 0
while(( float(1) + precision ) != float(1)):
    precision = precision/10
    i += 1

print(precision,i)

1.0000000000000001e-16 16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [ ]:
import math

def grad2solutions(a,b,c):
    squared = (b*b-4*a*c)**.5
    return [((-b) + squared)/(2*a),((-b) - squared)/(2*a)]

solution = grad2solutions(0.001,1000,0.001)

print(solution)

In [ ]:
def grad2solutions(a,b,c):
    squared = (b*b-4*a*c)**.5
    return [ (((-b) + squared)*(-b - squared))/(2*a*(-b - squared)),(((-b) - squared)*(-b + squared))/(2*a*(-b + squared))]

solution = grad2solutions(0.001,1000,0.001)

print(solution)


In [ ]:
import numpy as np

def grad2solutions1(a,b,c):
    squared = np.sqrt(b*b-4*a*c)
    return [((-b) + squared)/(2*a),((-b) - squared)/(2*a)]

solution1 = grad2solutions1(0.001,1000,0.001)

def grad2solutions2(a,b,c):
    squared = np.sqrt(b*b-4*a*c)
    return [ (((-b) + squared)*(-b - squared))/(2*a*(-b - squared)),(((-b) - squared)*(-b + squared))/(2*a*(-b + squared))]

solution2 = grad2solutions2(0.001,1000,0.001)

print(solution1,solution2)

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [ ]:
def myderivative(delta):
    return ((1 + delta)*(1 + delta - 1) - (1-1))/delta

deltas = [-2*x for x in range(2,8)]
#print(deltas)

for i in range(6):
    print(myderivative(10**deltas[i]))

print(myderivative(10**(-2)))

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [ ]:
import math

def areasemicircle(n):
    b = 2 / n
    x = -1
    area = 0
    
    while(x<1):
        area += ((1-(x**2))**.5)*b
        x += b
    
    return area

N = 4500000
%timeit print('For N=',N ,'The error is:', math.pi/2 - areasemicircle(N))

In [ ]:
import math

def areasemicircle(n):
    b = 2 / n
    x = -1
    area = 0
    
    while(x<1):
        area += ((1-(x**2))**.5)*b
        x += b
    
    return area

N = 240000000
%timeit print('For N=',N ,'The error is:', math.pi/2 - areasemicircle(N))


In [ ]:
# Apparently there's no gain, theoretically dividing the area can be beneficial because we get a 
# more accurate result, but dividing the area too much will sum all the small approximation errors resulting
# in a much bigger error (10 times!)